In [32]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import json
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#### Fetch xml sitemap

In [33]:
sitemap = pd.read_xml("https://www.rtbf.be/site-map/articles.xml")
sitemap.head()

,loc,lastmod,changefreq,news,image
0,https://www.rtbf.be/article/links-a-la-decouve...,2023-08-01T11:09:29.000Z,hourly,NaN,NaN
1,https://www.rtbf.be/article/escales-dans-l-uni...,2023-08-01T11:08:09.000Z,hourly,NaN,NaN
2,https://www.rtbf.be/article/direct-video-chine...,2023-08-01T11:07:05.000Z,hourly,NaN,NaN
3,https://www.rtbf.be/article/direct-video-haiti...,2023-08-01T11:06:11.000Z,hourly,NaN,NaN
4,https://www.rtbf.be/article/motorhead-les-cend...,2023-08-01T11:00:37.000Z,hourly,NaN,NaN


#### Keep only two first columns and change their name

In [34]:
df = sitemap.drop(["changefreq", "news", "image"], axis=1)
df.rename(columns={"loc": "source_url", "lastmod": "last_modified_date"}, inplace=True)
df.head()

,source_url,last_modified_date
0,https://www.rtbf.be/article/links-a-la-decouve...,2023-08-01T11:09:29.000Z
1,https://www.rtbf.be/article/escales-dans-l-uni...,2023-08-01T11:08:09.000Z
2,https://www.rtbf.be/article/direct-video-chine...,2023-08-01T11:07:05.000Z
3,https://www.rtbf.be/article/direct-video-haiti...,2023-08-01T11:06:11.000Z
4,https://www.rtbf.be/article/motorhead-les-cend...,2023-08-01T11:00:37.000Z


#### Keep only date in 'modified_date' column

In [35]:
df["last_modified_date"].replace({r"T.+": ""}, inplace=True, regex=True)
df.head()

,source_url,last_modified_date
0,https://www.rtbf.be/article/links-a-la-decouve...,2023-08-01
1,https://www.rtbf.be/article/escales-dans-l-uni...,2023-08-01
2,https://www.rtbf.be/article/direct-video-chine...,2023-08-01
3,https://www.rtbf.be/article/direct-video-haiti...,2023-08-01
4,https://www.rtbf.be/article/motorhead-les-cend...,2023-08-01


In [36]:
urls = df["source_url"]
urls.head()

0    https://www.rtbf.be/article/links-a-la-decouve...
1    https://www.rtbf.be/article/escales-dans-l-uni...
2    https://www.rtbf.be/article/direct-video-chine...
3    https://www.rtbf.be/article/direct-video-haiti...
4    https://www.rtbf.be/article/motorhead-les-cend...
Name: source_url, dtype: object

In [37]:
article_one = urls[0]
print(article_one)

https://www.rtbf.be/article/links-a-la-decouverte-de-groupes-alternatifs-et-de-leurs-influences-11227152


In [38]:
response = requests.get(article_one)
response.text

'<!DOCTYPE html><html lang="fr" dir="ltr"><head><meta charSet="utf-8"/><meta name="viewport" content="initial-scale=1.0, width=device-width, minimum-scale=1"/><meta property="al:android:url" content="rtbf://applinks/article/links-a-la-decouverte-de-groupes-alternatifs-et-de-leurs-influences-11227152"/><meta property="al:android:app_name" content="RTBF"/><meta property="al:android:package" content="be.rtbf"/><meta property="al:web:url" content="https://rtbf.be"/><link rel="preconnect" href="https://ds.static.rtbf.be"/><link rel="dns-prefetch" href="https://ds.static.rtbf.be"/><link rel="preconnect" href="https://ds1.static.rtbf.be"/><link rel="dns-prefetch" href="https://ds1.static.rtbf.be"/><link rel="preconnect" href="https://static-oaos.rtbf.be"/><link rel="dns-prefetch" href="https://static-oaos.rtbf.be"/><link rel="preconnect" href="https://sgc.static.rtbf.be"/><link rel="dns-prefetch" href="https://sgc.static.rtbf.be"/><link rel="preconnect" href="https://www.static.rtbf.be"/><lin

In [39]:
soup = bs(response.content, "html.parser")


#### Find article title

In [40]:
title = soup.find("h1").text
print(title)

Links : à la découverte de groupes alternatifs et de leurs influences


#### Find article title function

In [41]:
def find_article_title(url: str) -> str:
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    article_title = soup.find("h1").text
    return article_title

In [42]:
print(find_article_title(article_one))

Links : à la découverte de groupes alternatifs et de leurs influences


#### Find article text function

In [43]:
def find_article_text(url: str) -> str:
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    paragraphs = [p.text for p in soup.find_all("p", attrs={"class": None})]
    article_text = "".join(paragraphs)
    return article_text

In [44]:
print(find_article_text(article_one))

Envie de vous mettre plein de nouveautés Underground dans les oreilles ? Envie de trouvailles pas comme les autres ? Suivez le lien, Manu Debiève vous emmène à la découverte de groupes alternatifs récents en retissant leurs liens musicaux. De leur filiation artistique à leurs racines géographiques en passant par leurs potes de label ou leurs fans célèbres, découvrez tout ce qui les relie pour comprendre qui ils sont et comment ils "sonnent".Découvrez cette séquence en direct à 21h15 du mardi au vendredi dans Classic 21 Underground.Tous les sujets de l'article


#### Find published date

In [45]:

script = soup.find('script', {"type": "application/ld+json"})
data = json.loads(script.text, strict=False)
published_date = data['datePublished']
published_date


'2023-07-13T11:12:54+02:00'

#### find published date function

In [46]:
def find_published_date(url:str) -> str:
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    script = soup.find('script', {"type": "application/ld+json"})
    data = json.loads(script.text, strict=False)
    published_date = data['datePublished']
    return published_date

In [47]:
print(find_published_date(article_one))

2023-07-13T11:12:54+02:00


#### Add published_date column to dataframe

In [48]:
df['published_date'] = df['source_url'].apply(find_published_date)


#### Keep only date from published_date

In [49]:
df["published_date"].replace({r"T.+": ""}, inplace=True, regex=True)

#### Add article_title column to dataframe

In [51]:
df['article_title'] = df['source_url'].apply(find_article_title)

#### Add article_text column to dataframe

In [53]:
df['article_text'] = df['source_url'].apply(find_article_text)

#### Rearrange column order

In [55]:
df = df.loc[:, ['source_url', 'article_title', 'article_text', 'published_date', 'last_modified_date']]

In [57]:
df.to_csv('data/rtbf_articles.csv')

In [56]:
df.head()

,source_url,article_title,article_text,published_date,last_modified_date
0,https://www.rtbf.be/article/links-a-la-decouve...,Links : à la découverte de groupes alternatifs...,Envie de vous mettre plein de nouveautés Under...,2023-07-13,2023-08-01
1,https://www.rtbf.be/article/escales-dans-l-uni...,Escales dans l'univers de la musique contempor...,Le 13 août 2023Ma musique est le reflet de mes...,2023-08-01,2023-08-01
2,https://www.rtbf.be/article/direct-video-chine...,"Direct vidéo Chine - Angleterre : 0-1, les Ang...",Les Three Lions seraient-elles devenues des mo...,2023-08-01,2023-08-01
3,https://www.rtbf.be/article/direct-video-haiti...,"Direct vidéo Haïti - Danemark : 0-1, les Danoi...",La neuvième édition de la Coupe du monde fémin...,2023-07-18,2023-08-01
4,https://www.rtbf.be/article/motorhead-les-cend...,Motörhead : les cendres de Lemmy dispersées au...,C’est lors de la 32e édition du Wacken Open Ai...,2023-08-01,2023-08-01
